In [2]:
import os
import shutil
import numpy as np
import torch
from facenet_pytorch import InceptionResnetV1
import torchvision.models as models
import torchvision.transforms as transforms
from PIL import Image
from tqdm import tqdm
from sklearn.cluster import KMeans
from sklearn.metrics import silhouette_score
import glob
import pickle
import warnings

warnings.filterwarnings("ignore")
device = torch.device("cuda" if torch.cuda.is_available() else "mps" if torch.backends.mps.is_available() else "cpu")
version = "0.1"

In [11]:
class ImageClusteringAlgorithm:
    def __init__(self):
        self.facenet_model = InceptionResnetV1(pretrained="vggface2").eval().to(device)
        self.resnet_model = models.resnet50(pretrained=True).eval().to(device)

    def compute_features(self, image_directory):
        self.image_directory = os.path.expanduser(image_directory)
        file_extensions = ("*.jpg", "*.jpeg", "*.png", "*.bmp", "*.tiff", "*.tif")
        self.image_paths = []
        for extension in file_extensions:
            self.image_paths.extend(glob.glob(os.path.join(self.image_directory, extension)))
        features = []
        for image_path in tqdm(self.image_paths, desc="Computing features..."):
            feature = self._generate_image_features(image_path)
            features.append(feature)
        self.features = np.vstack(features)

    def compute_clusters(self, max_clusters, find_optimal_num_clusters=True):
        max_score = 0
        min_clusters = max_clusters
        if find_optimal_num_clusters:
            min_clusters = 2
        for i in tqdm(range(min_clusters, max_clusters + 1), "Calculating optimal clusters..."):
            kmeans = KMeans(n_clusters=i, random_state=42)
            labels = kmeans.fit_predict(self.features)
            silhouette_avg = silhouette_score(self.features, labels)
            print(f"Silhouette score: {silhouette_avg:.4f}")
            if silhouette_avg > max_score:
                print(f"new best cluster: {i}, silhouette score: {silhouette_avg:.4f}")
                max_score = silhouette_avg
                self.num_clusters = i
                self.labels = labels
                self.clusters = [[] for _ in range(self.labels.max() + 1)]
                for idx, label in enumerate(self.labels):
                    self.clusters[label].append(self.image_paths[idx])

    def save_clustered_images(self, output_directory, dry_run=False):
        output_directory = os.path.expanduser(output_directory)
        os.makedirs(output_directory, exist_ok=True)
        for cluster_id in tqdm(range(self.num_clusters), "Saving clusters..."):
            i = 0
            cluster_name_zfill = len(str(self.num_clusters))
            cluster_name = format(f"{cluster_id}".zfill(cluster_name_zfill))
            cluster_element_zfill = len(str(f"{len(self.clusters[cluster_id])}"))
            for image_path in self.clusters[cluster_id]:
                cluster_element = format(f"{i}".zfill(cluster_element_zfill))
                new_file_name = os.path.join(output_directory, f"cluster_{cluster_name}_{cluster_element}{os.path.splitext(image_path)[1]}")
                i = i + 1
                if dry_run:
                    print(new_file_name)
                else:
                    shutil.copyfile(image_path, new_file_name)

    def _generate_image_features(self, image_path):
        image = Image.open(image_path).convert("RGB")
        transform = transforms.Compose([transforms.Resize((224, 224)), 
                                        transforms.ToTensor(), 
                                        transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])])
        image_tensor = transform(image).unsqueeze(0).to(device)
        with torch.no_grad():
            resnet_features = self.resnet_model(image_tensor)
            face_features = self.facenet_model(image_tensor)
        features = np.concatenate((face_features.cpu().detach().numpy(), resnet_features.cpu().detach().numpy()), axis=1)
        return features
    
    def save_features(self, file_name):
        file_name = os.path.expanduser(file_name)
        data = { "features": self.features, "image_paths": self.image_paths, "image_directory": self.image_directory, "version": version }
        with open(file_name, "wb") as file: 
            pickle.dump(data, file)
    
    def load_features(self, file_name):
        file_name = os.path.expanduser(file_name)
        with open(file_name, "rb") as file:
            data = pickle.load(file)
        self.features = data["features"]
        self.image_paths = data["image_paths"]
        self.image_directory = data["image_directory"]

    def save_clusters(self, file_name):
        file_name = os.path.expanduser(file_name)
        data = { "num_clusters": self.num_clusters, "clusters": self.clusters, "version": version }
        with open(file_name, "wb") as file: 
            pickle.dump(data, file)

    def load_clusters(self, file_name):
        with open(file_name, "rb") as file:
            data = pickle.load(file)
        self.num_clusters = data["num_clusters"]
        self.clusters = data["clusters"]

In [17]:
ica = ImageClusteringAlgorithm()
image_directory = os.path.expanduser("~/Downloads/JPGs")
features_file = os.path.expanduser("~/Downloads/JPGs/Features_v0.1.pkl")
clusters_file = os.path.expanduser("~/Downloads/JPGs/Clusters_v0.1.pkl")
if not os.path.exists(features_file):
    ica.compute_features(image_directory)
    ica.save_features(features_file)
else:
    ica.load_features(features_file)
if not os.path.exists(clusters_file):
    max_clusters = 100
    ica.compute_clusters(max_clusters)
    ica.save_clusters(clusters_file)
else:
    ica.load_clusters(clusters_file)
#ica.save_clustered_images("~/Downloads/ClusteredJPGs")

Calculating optimal clusters...:   1%|          | 1/99 [00:05<08:30,  5.21s/it]

Silhouette score: 0.1132
new best cluster: 2, silhouette score: 0.1132


Calculating optimal clusters...:   2%|▏         | 2/99 [00:11<09:09,  5.66s/it]

Silhouette score: 0.0914


Calculating optimal clusters...:   3%|▎         | 3/99 [00:17<09:27,  5.91s/it]

...

Silhouette score: 0.0547


Calculating optimal clusters...:  99%|█████████▉| 98/99 [38:00<00:39, 39.39s/it]

Silhouette score: 0.0537


Calculating optimal clusters...: 100%|██████████| 99/99 [38:39<00:00, 23.43s/it]

Silhouette score: 0.0557
